# Project Title: TLC Trip Records Data Exploration

Project Description: https://www.nyc.gov/assets/tlc/downloads/pdf/trip_record_user_guide.pdf https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_fhv.pdf https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

# Import Modules

In [1]:
import os
import sys
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

# Create Spark Session

In [2]:
spark = SparkSession.builder.appName('TLC Trip Record Data').getOrCreate()
print(f'The PySpark {spark.version} version is running....')

The PySpark 3.3.2 version is running....


In [3]:
df = spark.read.parquet('yellow_tripdata_2023-03.parquet')

In [4]:
df.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-03-01 01:06:43|  2023-03-01 01:16:43|              1|          0.0|         1|                 N|         238|          42|           2|        8.6|  1.0|    0.5|       0.